<a href="https://colab.research.google.com/github/DelmiroDaladier/NLP-studies/blob/master/machine_translation/machine_translation_with_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import numpy as np
import pandas as pd
from string import digits
from sklearn.model_selection import train_test_split

from keras import Model
from keras.models import load_model
from keras.layers import Dropout, Input, Embedding, LSTM, Dense

In [29]:
en_dataset = pd.read_csv('small_vocab_en.csv', header=None, error_bad_lines=False, sep='/n')
fr_dataset = pd.read_csv('small_vocab_fr.csv', header=None, error_bad_lines=False, sep='/n')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [30]:
en_dataset.rename({0:'text'}, inplace=True, axis=1)
fr_dataset.rename({0:'text'}, inplace=True, axis=1)


In [31]:
remove_digits =  str.maketrans('', '', digits) 

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.lower())
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.lower())

en_dataset['text'] = en_dataset['text'].apply(lambda x: re.sub("[^\w\s]", "",x))
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: re.sub("[^\w\s]", "",x))

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.translate(remove_digits))
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.translate(remove_digits))

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.strip())
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.strip())

en_dataset['text'] = en_dataset['text'].apply(lambda x: '<SOS> ' + x + ' <EOS>')
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: '<SOS> ' + x + ' <EOS>')

In [32]:
word_to_index_en = {}
index_to_word_en = {}
word_index = 0
for sentence in en_dataset['text'].values:
  for word in sentence.split():
    if word not in word_to_index_en.keys():
      word_to_index_en[word] = word_index
      index_to_word_en[word_index] = word
      word_index += 1


In [33]:
max_len_en = max([len(sentence.split()) for sentence in en_dataset['text']])

In [34]:
word_to_index_fr = {}
index_to_word_fr = {}
word_index = 0
for sentence in fr_dataset['text'].values:
  for word in sentence.split():
    if word not in word_to_index_fr.keys():
      word_to_index_fr[word] = word_index
      index_to_word_fr[word_index] = word
      word_index += 1

In [35]:
max_len_fr = max([len(sentence.split()) for sentence in fr_dataset['text']])

In [36]:
num_encoder_tokens = len(word_to_index_en.keys())
num_decoder_tokens = len(word_to_index_fr.keys())

In [37]:
X = en_dataset['text'].values
Y = fr_dataset['text'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

In [39]:
def create_data_generator(x = X_train, y = Y_train, batch_size=128):

  while True:
    for j in range(0, len(x), batch_size):
      encoder_input_data = np.zeros((batch_size, max_len_en), dtype="float32")
      decoder_input_data = np.zeros((batch_size, max_len_fr), dtype="float32")
      decoder_target_data = np.zeros((batch_size, max_len_fr, num_decoder_tokens), dtype="float32")

      for i, (input_text, target_text) in enumerate(zip(x[j:j+batch_size], y[j:j+batch_size])):
        for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = word_to_index_en[word] 
        for t, word in enumerate(target_text.split()):
          if t < len(target_text.split())-1:
            decoder_input_data[i, t] = word_to_index_fr[word]
          if t > 0 :
            decoder_target_data[i, t - 1, word_to_index_fr[word]] = 1.0

      yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [40]:
latent_dim=64
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = decoder_embedding(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    dec_emb,
    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [41]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     12864       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     22144       input_4[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 128
epochs = 200

model.fit_generator(generator = create_data_generator(X_train, Y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = create_data_generator(X_test, Y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

model.save('machine_translation')

Epoch 1/200
969/969 [==============================] - 117s 121ms/step - loss: 0.0043 - acc: 0.9980 - val_loss: 0.0110 - val_acc: 0.9948
Epoch 2/200
539/969 [===============>..............] - ETA: 48s - loss: 0.0041 - acc: 0.9980

In [ ]:
model = load_model("machine_translation")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output  # lstm
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_hidden_state")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_cell_state")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb_layer = model.layers[3]
dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_lstm = model.layers[5] # lstm_1
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


In [ ]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1))

    target_seq[0, 0] = word_to_index_fr['<SOS>']
   
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =index_to_word_fr[sampled_token_index]
        decoded_sentence += ' '+ sampled_word

        if sampled_word == '<EOS>' or len(decoded_sentence.split()) > max_len_fr:
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

In [ ]:
train_gen = create_data_generator(X_test, Y_test, batch_size = 1)
for seq_index in range(3):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", X_test[seq_index])
    print("Real sentence:", Y_test[seq_index])
    print("Predicted sentence:", decoded_sentence)

In [ ]:
a, b = next(train_gen)
print(b.shape)

In [20]:
latent_dim=128